#Universidade Presbiteriana Mackenzie
##Faculdade de Computação e Informática
##Programa de Pós-Graduação em Computação Aplicada
##Projeto Final da Disciplina Aprendizado de Máquina
##[Turma GER_ENST51827] - 2024/2
###Prof: Dr. Leandro Augusto da Silva
###Aluno: André Flaeschen
---

# Objetivos
O Trabalho consiste em analisar a base de dados de 2019 da Polícia Rodoviária Federal Brasileira sobre acidentes em rodovias federais para identificar as principais causas, condições e localidades das ocorrências. A partir da análise, extrair insights importantes.

A base de dados de Acidentes Rodoviários disponibilizada no site da Kaglle:
 https://www.kaggle.com/datasets/apwinterjunior/acidentesprf-2017-2019?select=acidentes.csv

Uma cópia foi salva no meu Google drive para servir a este trabalho.


In [11]:
from matplotlib import pyplot as plt
#para contar a frequência das classes
from collections import Counter
#para manipulação de arrays
import numpy as np

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dataset_path = '/content/drive/MyDrive/Colab Notebooks/datasets/acidentes.csv'

import pandas as pd
df = pd.read_csv(dataset_path, sep=";")

print(df.head())

print("A base de dados original possui a quantidade total de registros:", len(df))


Mounted at /content/drive
      Nome da Origem    id   pesid data_inversa dia_semana   horario  uf  \
0  acidentes2017.csv   8.0     1.0   2017-01-01    domingo  00:00:00  PR   
1  acidentes2017.csv   9.0   955.0   2017-01-01    domingo  00:01:00  SC   
2  acidentes2017.csv  11.0     3.0   2017-01-01    domingo  00:00:00  PR   
3  acidentes2017.csv  11.0     2.0   2017-01-01    domingo  00:00:00  PR   
4  acidentes2017.csv  12.0  1499.0   2017-01-01    domingo  00:00:00  GO   

      br    km                 municipio  ...       sexo ilesos feridos_leves  \
0  376.0   112                 PARANAVAI  ...  Masculino      0             0   
1  101.0   234                   PALHOCA  ...  Masculino      1             0   
2  153.0  56,9  SANTO ANTONIO DA PLATINA  ...  Masculino      0             1   
3  153.0  56,9  SANTO ANTONIO DA PLATINA  ...   Feminino      0             1   
4  153.0   435                  ANAPOLIS  ...  Masculino      0             0   

  feridos_graves mortos      l

# Seleção dos dados
A partir da base original, faremos a seleção apenas das ocorrências do ano de 2019 e salvaremos um novo dataset.

In [6]:
df_acidentes2019 = df[df['Nome da Origem'] == 'acidentes2019.csv']

# Salvando o novo dataset em um arquivo CSV
df_acidentes2019.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/acidentes2019.csv', index=False, sep=';')

print("Novo dataset criado com sucesso!")

print(df_acidentes2019.head())
print("Quantidade total de registros para o ano de 2019:", len(df_acidentes2019))


Novo dataset criado com sucesso!
           Nome da Origem        id     pesid data_inversa   dia_semana  \
369197  acidentes2019.csv  182256.0  403856.0   2019-01-01  terça-feira   
369198  acidentes2019.csv  182263.0  402859.0   2019-01-01  terça-feira   
369199  acidentes2019.csv  182277.0  402850.0   2019-01-01  terça-feira   
369200  acidentes2019.csv  182289.0  402431.0   2019-01-01  terça-feira   
369201  acidentes2019.csv  182307.0  402642.0   2019-01-01  terça-feira   

         horario  uf     br     km         municipio  ...       sexo ilesos  \
369197  04:00:00  CE  116.0  136,9            RUSSAS  ...  Masculino      1   
369198  05:00:00  MT  158.0  599,5          AGUA BOA  ...  Masculino      1   
369199  10:00:00  PA   10.0     34        DOM ELISEU  ...  Masculino      1   
369200  08:30:00  BA  101.0     13          RIO REAL  ...  Masculino      1   
369201  13:50:00  BA  116.0    440  FEIRA DE SANTANA  ...  Masculino      0   

       feridos_leves feridos_graves morto

Selecionando apenas as colunas de interesse para o estudo.

In [9]:
colunas_interesse = ['id','data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'marca', 'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos']

df_filtrado = df_acidentes2019[colunas_interesse]

print(df_filtrado.head())

              id data_inversa   dia_semana   horario  uf     br     km  \
369197  182256.0   2019-01-01  terça-feira  04:00:00  CE  116.0  136,9   
369198  182263.0   2019-01-01  terça-feira  05:00:00  MT  158.0  599,5   
369199  182277.0   2019-01-01  terça-feira  10:00:00  PA   10.0     34   
369200  182289.0   2019-01-01  terça-feira  08:30:00  BA  101.0     13   
369201  182307.0   2019-01-01  terça-feira  13:50:00  BA  116.0    440   

               municipio               causa_acidente  \
369197            RUSSAS             Animais na Pista   
369198          AGUA BOA  Defeito Mecânico no Veículo   
369199        DOM ELISEU      Velocidade Incompatível   
369200          RIO REAL           Ingestão de Álcool   
369201  FEIRA DE SANTANA           Ingestão de Álcool   

                    tipo_acidente  ...                     marca  \
369197    Atropelamento de Animal  ...  I/MB 15SPRINT RONTAN AMB   
369198                   Incêndio  ...          FORD/CARGO 815 N   
369199  